In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm

from datetime import datetime

import warnings
warnings.simplefilter('ignore')

In [2]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [3]:
# 検証期間 JPX日経中小型株指数連動ETF上場 ~ 2022/7/29まで 1300営業日
start = datetime(2017, 4, 3)
end = datetime(2022, 8, 1)

In [4]:
nikkei = yf.download('^N225', start=start, end=end, adjusted=True)
nikkei.index = nikkei.index.tz_localize(None)
nikkei = nikkei[(nikkei.index>=start) & (nikkei.index<end)].sort_index()[['Adj Close']]
nikkei = nikkei.rename(columns={'Adj Close':'Nikkei225'})

[*********************100%***********************]  1 of 1 completed


In [5]:
# MAXIS JPX日経インデックス400上場投信
nk400 = yf.download('1593.T', start=start, end=end, adjusted=True)
nk400.index = nk400.index.tz_localize(None)
nk400 = nk400[(nk400.index>=start) & (nk400.index<end)].sort_index()[['Adj Close']]
nk400 = nk400.rename(columns={'Adj Close':'Index'})
nk400 = pd.concat([nikkei, nk400['Index']], axis=1, join='inner')

[*********************100%***********************]  1 of 1 completed


In [6]:
# MAXIS JPX日経中小型株指数上場投信
nk200 = yf.download('1492.T', start, end, adjusted=True)
nk200.index = nk200.index.tz_localize(None)
nk200 = nk200[(nk200.index>=start) & (nk200.index<end)].sort_index()[['Adj Close']]
nk200 = nk200.rename(columns={'Adj Close':'Index'})
nk200 = pd.concat([nikkei, nk200], axis=1, join='inner')

[*********************100%***********************]  1 of 1 completed


In [7]:
# CSVを読み込み、指数構成銘柄のリストを作成
nk400_code = pd.read_csv('../constituent_stocks/jpx_nikkei_index_400_weight_jp.csv', encoding='shift-jis')['コード'][:-1].tolist()
nk200_code = pd.read_csv('../constituent_stocks/jpx_nikkei_mid_small_weight_jp.csv', encoding='shift-jis')['コード'][:-1].tolist()

In [8]:
# 個別銘柄の株価を取得してリストに格納する関数
def list_company_stock_price(code, df_list):
    df = yf.download(f'{int(code)}.T', start, end, adjusted=True, progress=False)[['Adj Close']]
    df.index = df.index.tz_localize(None)
    df = df.rename(columns={'Adj Close':code})
    df_list.append(df)

In [9]:
# 個別銘柄の株価を格納する list
nk400_df_list = []
nk200_df_list = []

In [10]:
# 日経400 構成銘柄の株価を取得してリストに格納
for code in tqdm(nk400_code):
    list_company_stock_price(code, nk400_df_list)

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [02:51<00:00,  2.33it/s]


In [11]:
# 日経中小型 構成銘柄の株価を取得してリストに格納
for code in tqdm(nk400_code):
    list_company_stock_price(code, nk200_df_list)

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [02:33<00:00,  2.60it/s]


In [12]:
# 結合して一つの DataFrame に
nk400_dfs = pd.concat(nk400_df_list, axis=1)
nk200_dfs = pd.concat(nk200_df_list, axis=1)

In [13]:
# インデックスと結合
nk400 = pd.merge(nk400, nk400_dfs, left_index=True, right_index=True, how='left')
nk200 = pd.merge(nk200, nk200_dfs, left_index=True, right_index=True, how='left')

In [14]:
nk400.to_csv('../stock_data/nk400.csv')
nk200.to_csv('../stock_data/nk200.csv')